# Scopus API Pull and Research Connection Script

#### <b> Last Updated: 2024-04-22 </b>
This is a script originally created in October 2019, and reconfigured in March 2024. It takes in Scopus IDs from WMU and WMed researchers and returns author keywords associated with their publications. These keywords, along with ingested keywords from ORCiD profiles, directory pages, departmental research topic websites, and FARS keywords, are then exported in as a csv aggregating researchers by a single research keyword (in the format "keyword | researcherID(s) | researchName(s) | researcherAffiliation(s)"). The overarching purpose of this project is to help make connections between researchers with similar interests for expanding interdisciplinary collaboration. The script accomplishes the following actions within each cell:

* <b>In[1]</b> Imports relevant libraries.\
<i>Note: pybliometrics is not a default python library and will need to be installed first using "pip install pybliometrics". Scopus API key will need to be acquired through the Elsevier Developer Portal at http://dev.elsevier.com/myapikey.html, which pybliometrics will prompt you to enter upon library installation to create a configuration file on your system. A InstToken is not required. To obtain the API Key, you will need to log into Scopus with your institutional affiliation. To query the API, users will need to be on campus.</i>
For more information about pybliometrics, please see the documentation: https://pybliometrics.readthedocs.io/en/stable/
<br></br>
* <b>In[2]</b> Lists all the .txt and .csv files required within the script. \
These include: a plain text (.txt) file listing all the researcher Scopus IDs, separated by commas (for querying the Scopus API, in the format: "ScopusID1,ScopusID2...etc"); a plain text file listing WMU researcher Scopus IDs (for querying the Scopus API, in the format: "ScopusID1,ScopusID2...etc"); a comma separated value (.csv) file created by the script in ln[4] that includes a full output of articles along with all the associated metadata for each researcher Scopus ID; a plain text file matching unique researcher IDs to related Scopus IDs (in the format "ID:ScopusID1; ScopusID2"); a plain text file matching unique researcher IDs to additional research keywords (in the format "ID:keywordORCiD; keywordDirectory; keywordFARS"); a plain text file making unique researcher IDs to researcher names and affiliation (in the format "ID:researcherName;Dept1 | Dept2").
<br></br>
* <b>In[3]</b> Creates list of ScopusIDs in a format readable by the API. \
Opens the listing of all the researcher Scopus IDs (researcher_ScopusID_list) and reads the contents in, modifying the IDs to include the designation "Au_ID" required by ScopusSearch (in ln[4]), and splitting the IDs into items within a list, authors_ids
<br></br>
* <b>In[4]</b> Returns articles associated with the ScopusIDs. \
Using the authors_ids list, all the researcher IDs are looped through the Scopus API via ScopusSearch, returning all the articles associated with each Scopus ID. These articles are added to a pandas dataframe, which holds onto the information while the next Scopus ID within the list is used to query the API. The resulting dataframe is exported ("date_researcher-articles-complete.csv"), using the date that the script was initialized
<br></br>
* <b>In[5]</b> Creates list of WMU ScopusIDs in a format readable by the API. \
Opens the listing of just WMU researcher Scopus IDs (partialresearcher_ScopusID_list) and reads the contents in, modifying the IDs to include the designation "Au_ID" required by ScopusSearch, and splitting the IDs into items within a list, partial_authors_ids
<br></br>
* <b>In[6]</b> Returns articles associated with only WMU ScopusIDs. \
Using the partial_authors_ids list, only the WMU researcher IDs are looped through the Scopus API via ScopusSearch, returning all the articles associated with each Scopus ID. These articles are added to a pandas dataframe, which holds onto the information while the next Scopus ID within the list is used to query the API. The resulting dataframe is exported ("date_researcher-articles-WMU.csv"), using the date that the script was initialized
<br></br>
* <b>In[7]</b> Uses script-generated file of all articles associated with ScopusIDs (exported in In[4]). \
The csv with the full Scopus ID articles list ("date_researcher_articles_list.csv") is read back into the script as a dataframe outside of querying the API. The dataframe is printed as a check to see if everything is working correctly
<br></br>
* <b>In[8]</b> Extracts author keywords from associated ScopusIDs in file (In[7]), merging all keywords across multiple scholarship under one ScopusID. \
A dictionary (aukey_dict) is created from the df_full_articles dataframe, where the Scopus ID (key) is associated with the author keywords (values) from all the articles associated with the Scopus ID. Since author keywords is a user-generated field, it is possible that a researcher may have many articles without any author keywords, and therefore the value for that ID key might be "nan" (none). It is also possible that a researcher might have many Scopus IDs if they haven't been merged, and will be represented more than once within this dictionary.
<br></br>
* <b>In[9]</b> Replaces ScopusIDs with researcherIDs, merging all ScopusIDs associated with a researcherID into a single entry. \
A txt file associating researcher IDs with their Scopus IDs is read into the script (researcher_scopus). A dictionary, researkey_dict, is created substituting the Scopus ID with researcher IDs as the key, and adding the Scopus author keywords as the values. In the case where a researcher does not have a Scopus ID, then the researcher ID is added to the dictionary without associated values.
<br></br>
* <b>In[10]</b> Adds additional research keywords to single researcherID entry. \
A txt file associating researcher IDs with additional keywords from ORCiD, directories, researcher departmental topic websites, Faculty Annual Reporting portal (FARS) is read into the script(researcher_additionalkeywords). The researkey_dict is used as a base to add these additional keywords to Scopus author keyword values already in the dictionary. After, the keywords (values) are de-duplicated. This step also generates an intermediary file (date_WMUresearcher_keywords_affiliation.csv), that exports all the keywords associated with a single researcher.
<br></br>
* <b>In[11]</b> Associates single research key with a single researcherID. \
The researkey_dict is then transformed into singleresearkey_dict, where the research keyword list is split up into single units, and each keyword is associated with a researcher ID (W567898:[1,2,3] --> W567898:[1], W567898:[2], W567898:[3])
<br></br>
* <b>In[12]</b> Associate single research key with multiple researchIDs, and adds additional context to each researchID. \
A new txt file is read into the script that associates researcher IDs with researcher name and WMU department | WMed department (researcher_affiliation). This is transformed into the uniqueIDaffil_dict, which is then used to add context to the keyresearaffil_dict, which links keywords to researcher IDs, researcher names, and researcher departments. All of the affiliation values are de-duplicated. Keyresear_dict is then transformed into a dataframe (keyresearaffil_dataframe) and the final file is exported (date_WMUresearcher_keywords_affiliation.csv) with a single keyword associated with multiple researchers.

Listing python libraries used within script

In [1]:
#used for querying the ScopusAPI
import pybliometrics 
from pybliometrics.scopus import ScopusSearch

#used to obtain today's date for file export
from datetime import date

#used to create dataframe for csv export
import pandas as pd 

Listing files used as part of script

In [2]:
researcher_ScopusID_list = "ScopusID_complete-list.txt"
partialresearcher_ScopusID_list = "ScopusID_WMU-list.txt" 
researcher_articles_list = "2024-04-17_researcher-articles-complete.csv"
researcher_scopus = "researcherID_scopusID.txt"
researcher_additionalkeywords = "researcherID_additionalkeywords.txt"
researcher_affiliation = "researcherID_researcherdept.txt"

Initiating ScopusAPI pull for complete researcher ScopusIDs (WMU and Wmed)

In [3]:
#read in list with Scopus IDs from all researchers (WMU and Wmed)
IDs = open(researcher_ScopusID_list, 'r')
read_IDs = IDs.read().split(',')

#creating list adding AU-ID extension to the front of each ID, required for ScopusSearch
authors_ids = []
for id in read_IDs:
    new = "AU-ID(" + id + ")"
    authors_ids.append(new)

In [4]:
#initializing a dataframe outside the loop that will match the expected shape of the scopus pull
s = ScopusSearch(authors_ids[0])
df_articles_composite = pd.DataFrame(pd.DataFrame(s.results))

#looping through each ScopusID in list
for index, ID in enumerate(authors_ids):
    s = ScopusSearch(ID) #look up each ID in Scopus
    df_results = pd.DataFrame(pd.DataFrame(s.results)) #get all scholarship metadata associated with each ID
    if index != 0: #if scholarship is associated with ID
        df_articles_composite = pd.concat([df_articles_composite, df_results]) #concatenating to composite dataframe
    
#looking up today's date and exporting composite dataframe to comma separated value file with utf-8 encoding
todays_date = date.today()
df_articles_composite.to_csv(f"{todays_date.isoformat()}_researcher-articles-complete.csv", index=False, encoding='utf-8-sig')

Initiating ScopusAPI pull for WMU researcher ScopusIDs only

In [5]:
#read in list with Scopus IDs from WMU only (excluding WMed)
partialIDs = open(partialresearcher_ScopusID_list, 'r')
read_partialIDs = partialIDs.read().split(',')

#creating list adding AU-ID extension to the front of each ID, required for ScopusSearch
partial_authors_ids = []
for id in read_partialIDs:
    new = "AU-ID(" + id + ")"
    partial_authors_ids.append(new)

In [6]:
#initializing a dataframe outside the loop that will match the expected shape of the scopus pull
s = ScopusSearch(partial_authors_ids[0])
df_partialarticles_composite = pd.DataFrame(pd.DataFrame(s.results))

#looping through each ScopusID in list
for index, ID in enumerate(partial_authors_ids):
    s = ScopusSearch(ID) #look up each ID in Scopus
    df_results = pd.DataFrame(pd.DataFrame(s.results)) #get all scholarship metadata associated with each ID
    if index != 0: #check if there is scholarship associated with ScopusID
        df_partialarticles_composite = pd.concat([df_partialarticles_composite, df_results]) #concatenate to composite dataframe

#looking up today's date and exporting partial composite dataframe to comma separated value file with utf-8 encoding    
todays_date = date.today()
df_partialarticles_composite.to_csv(f"{todays_date.isoformat()}_researcher-articles-WMU.csv", index=False, encoding='utf-8-sig')

Importing back in the file of all articles associated with WMU and Wmed researchers indexed by Scopus

In [7]:
df_full_articles = pd.read_csv(researcher_articles_list,encoding='utf-8-sig')
df_full_articles #preview metadata associated with each piece of scholarship

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85141012738,10.1177/10982140211062884,NaN,NaN,Thinking Outside the Self-Report: Using Evalua...,ar,Article,Wingate L.A.,60001439;60000879;101288518,Pennsylvania State University;Western Michigan...,...,515-538,"In this study, we investigated the impact of t...",evaluation capacity building | evaluation trai...,0,0,NaN,NaN,NSF,1600992,National Science Foundation
1,2-s2.0-85138312850,NaN,NaN,NaN,Creating a collaborative cross-institutional c...,cp,Conference Paper,Cervato C.,128584608,Chemical Engineering,...,NaN,NSF ADVANCE has been instrumental in supportin...,NaN,2,0,NaN,NaN,NSF,1935932,National Science Foundation
2,2-s2.0-85150569347,10.1016/B978-0-12-818630-5.09052-7,NaN,NaN,Metaevaluation: key concepts explained,ch,Book Chapter,Wingate L.A.,60000879,Western Michigan University,...,263-274,This article's purpose is to acquaint readers ...,Evaluation quality | Evaluation standards | Me...,0,0,NaN,NaN,NaN,undefined,NaN
3,2-s2.0-85050960472,10.1016/j.evalprogplan.2018.06.004,S0149718917302331,30092490.0,The project vita: A dynamic knowledge manageme...,ar,Article,Wingate L.A.,60030551;60000879,Syracuse University;Western Michigan University,...,22-27,A project vita is a comprehensive index of fac...,Accountability | Center | Evaluation | Knowled...,7,1,publisherfree2read,Bronze,NSF,1600992,National Science Foundation
4,2-s2.0-85138417336,10.1017/9781107477261.011,NaN,NaN,Monitoring and Evaluation,ch,Book Chapter,MacDonald G.,60021658;60000879,Centers for Disease Control and Prevention;Wes...,...,122-135,NaN,NaN,0,0,NaN,NaN,NaN,undefined,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21792,2-s2.0-67649342663,10.5408/1089-9995-56.3.251,NaN,NaN,Geoscience conceptual knowledge of preservice ...,ar,Article,Petcovic H.L.,60000879,Western Michigan University,...,251-260,Effective instruction hinges in part on unders...,NaN,25,0,NaN,NaN,NaN,undefined,NaN
21793,2-s2.0-37349131275,10.1029/2007EO420009,NaN,NaN,Geoscience faculty discuss courses for future ...,ar,Article,Manduca C.,60013250;60003631;60000879,"Carleton College, USA;Western Washington Unive...",...,428,NaN,NaN,0,1,publisherfree2read,Bronze,NaN,undefined,NaN
21794,2-s2.0-29444432507,10.1029/2004JB003432,NaN,NaN,Modeling magma flow and cooling in dikes: Impl...,ar,Article,Petcovic H.,60015481;60000879,University of Washington;Western Michigan Univ...,...,1-15,The Columbia River flood basalts include some ...,NaN,65,0,NaN,NaN,NaN,undefined,NaN
21795,2-s2.0-85046234249,10.1130/0-8137-0004-3.87,NaN,NaN,The Columbia River flood basalts and the Yakim...,ar,Article,Reidel S.P.,60023471;60016326;60015120;60013402,Pacific Northwest National Laboratory;Washingt...,...,87-105,This field trip guide covers a two-day trip to...,NaN,16,0,NaN,NaN,NaN,undefined,NaN


Using the full Scopus article file to match rows on ScopusIDs, and then extracting author keywords associated with each row to create a dictionary associating ScopusID to author keywords within indexed articles

In [8]:
#creating ScopusID:authorkeyword dictionary outside of loop
aukey_dict = {}

#removing AU-ID extension from the front of each ScopusID and saving to new list
stripped_author_ids = [id.strip("AU-ID(").strip(")") for id in authors_ids]

#creating placeholder keyword list to concatenate keywords across articles for each ScopusID
keyword_list = []
#looping through each ID in newly created list
for id in stripped_author_ids:
    id_matched_rows = df_full_articles['author_ids'].str.contains(id) #checking ScopusID in list against ScopusID in row
    id_matched_keywords = df_full_articles[id_matched_rows]['authkeywords'] #grabbing author keywords from matched row
    massaged_keywords_list_list = id_matched_keywords.str.replace(" | ",",",regex=False).str.lower().str.split(",") #changing author keywords from string to list
    for row in massaged_keywords_list_list:
        if row: #checking if the row is not an empty list (an empty list in python is considered False)
            if type(row) == type([]): #if the row is empty, the id didn't match any articles
                #if the id did match an article, but there were no keywords provided then the result is nan (which would cause an error with the extend method)
                keyword_list.extend(row) #adding keywords to placeholder keyword list
    aukey_dict[id] = keyword_list #creating key:value pairs in dictionary
    keyword_list = [] #clearing placeholder list
    
aukey_dict #preview as proof of concept

{'9846499400': ['evaluation capacity building',
  'evaluation training',
  'interrupted time series',
  'professional development',
  'evaluation quality',
  'evaluation standards',
  'meta-evaluation',
  'metaevaluation',
  'metaevaluation criteria',
  'metaevaluation methods',
  'program evaluation',
  'accountability',
  'center',
  'evaluation',
  'knowledge management',
  'research and development',
  'resume',
  'vita',
  'evaluation',
  'formative evaluation',
  'government auditing standards',
  'guiding principles for evaluators',
  'meta-evaluation',
  'program evaluation',
  'program evaluation standards',
  'summative evaluation',
  'evaluation',
  'formative evaluation',
  'government auditing standards',
  'guiding principles for evaluators',
  'meta-evaluation',
  'program evaluation',
  'program evaluation standards',
  'summative evaluation',
  'evaluation quality',
  'evaluation standards',
  'meta-evaluation',
  'metaevaluation',
  'metaevaluation criteria',
  'metae

Replacing ScopusID as the main identifier within the dictionary with researcherID, a unique ID assigned to researchers by the creator outside the script. This new dictionary also creates an identifier with a blank list for those who might not have scholarship indexed by Scopus or might not have author keywords associated with their articles within Scopus.

In [9]:
#creating researcherID:Scopuskeyword dictionary outside of loop
researkey_dict = dict({})

#looping through each researcherID:scopusID(s) list
with open(researcher_scopus, 'r', encoding="utf8") as f: #opening file
    uniqueID_scopusID = f.readlines() #reading in entire file as string
    for line in uniqueID_scopusID: #for each researcherID and ScopusID(s) pairing within line
        temp_list = line.rstrip('\n').replace(';',':').split(':') #split line of string into list
        uniqueID = temp_list[0] #take the first value in list (researcherID) and put it into a temporary list
        
        if temp_list[1] != '': #if there are ScopusIDs associated with the researcherID
            scopusIDs = temp_list[1:] #put the rest of the list values (ScopusIDs) into a temporary list
            
            for ID in scopusIDs: #start looping through ScopusIDs in list of all researcher ScopusIDs
                if ID in aukey_dict: #check if ID is in ScopusID:authorkeyword dictionary
                    if not uniqueID in researkey_dict: #if researcherID is not in researcherID:Scopuskeyword dictionary
                        if type(aukey_dict[ID]) is float: #check if there are no author keywords associated with ScopusID (values in dictionary are nan (float))
                            researkey_dict[uniqueID] = [] #append blank list as value to the researcherID key in dictionary
                        else: #if there are author keywords associated with ScopusID
                            researkey_dict[uniqueID] = aukey_dict[ID] #add those keywords as values to the researcherID key in dictionary
                    else: #if researcherID is already in researcherID:Scopuskeyword dictionary
                        if type(aukey_dict[ID]) is float: #check if there are no author keywords associated with ScopusID (values in dictionary are nan (float))
                            pass #if there are no author keywords, do nothing
                        else: #if there are author keywords associated with ScopusID
                            researkey_dict[uniqueID].extend(aukey_dict[ID]) #add those author keywords to values in the researcherID key in dictionary
                else: #if ID is not in ScopusID:authorkeyword dictionary
                    if not uniqueID in researkey_dict: #and if researcherID is already not in researcherID:Scopuskeyword dictionary
                        researkey_dict[uniqueID] = [] #add the researcherID to dictionary with blank list as value
                        
        else: #if there are no ScopusIDs associated with the researcherID
            researkey_dict[uniqueID] = [] #add the researcherID to dictionary with blank list as value

Adding additional keywords from sources outside of Scopus, scraped from ORCiD profiles, directory researcher keywords or biographies, landing pages for departmental research topics, and from the Faculty Annual Reporting System (FARS), to each of the researcherIDs. Deduplicating keywords to include only unique values. Exporting intermediary file that lists all keywords for each researcher (before key-pairs are flipped in the next step).

In [10]:
with open(researcher_additionalkeywords, 'r', encoding="utf8") as f: #opening file
    uniqueID_keywords = f.readlines() #reading in entire file as string
    for line in uniqueID_keywords: #for each researcherID and keywords pairing within line
        temp_list = line.rstrip('\n').replace('; ',':').split(':') #split line of string into list
        uniqueID = temp_list[0] #take the first value in list (researcherID) and put it into a temporary list
        
        if temp_list[1] != '': #if there are additional keywords associated with the researcherID
            keywords = temp_list[1:] #put the rest of the list values (additional keywords) into a temporary list
            researkey_dict[uniqueID].extend(keywords) #then add these additional keywords to the researcherID:Scopuskeywords dictionary
        researkey_dict[uniqueID] = list(set(researkey_dict[uniqueID])) #dedeplicate the keyword values
        
researkey_dict #preview dictionary to see if additional keywords appended and deduplicated correctly

#creating dataframe from dictionary, where the researcherID is one column and keywords are a second column
researkey_dataframe = pd.DataFrame.from_dict(researkey_dict, orient='index')

#looking up today's date and exporting partial composite dataframe to comma separated value file with utf-8 encoding    
todays_date = date.today()
researkey_dataframe.to_csv(f"{todays_date.isoformat()}_WMUresearcher_keywords_check.csv", index=True, encoding='utf-8-sig')

Flipping key (researcherID) and values (research keywords) so that researchers are aggregated under shared keywords (keyword:researcher(s))

In [11]:
#creating keyword:researcherID(s) dictionary outside of loop
keyresear_dict = dict({})

#looping through each entry within the researcherID:keyword dictionary
for uniqueID, keywords in researkey_dict.items():
    for keyword in keywords: #for each individual keyword within the keyword list
        if keyword in keyresear_dict: #check if the keyword is already in the keyword:researcherID(s) dictionary
            keyresear_dict[keyword].append(uniqueID) #if the keyword exists, add researcherID to it
        else: #if the keyword does not already exist within the dictionary
            keyresear_dict[keyword] = [uniqueID] #create a new keyword key entry and add affiliated researcherID as value

#checking to see if multiple researcherIDs added correctly to keyword
for keyword, IDs in keyresear_dict.items():
    if len(IDs) > 1: #only selecting keywords that have more than one associated researcherID
        print(keyword, IDs)

covid-19 ['W0001', 'W0179', 'W0215', 'W0216', 'W0221', 'W0222', 'W0242', 'W0248', 'W0249', 'W0274', 'W0342', 'W0343', 'W0349', 'W0350', 'W0713', 'W0740', 'W0823', 'W0831', 'W0891', 'W1012', 'W1013', 'W1034', 'W1040', 'W1085', 'W1093', 'W1102', 'W1124', 'W1167', 'W1171', 'W1336', 'W1343', 'W1364', 'W1423', 'W1427', 'W1429', 'W1446', 'W1464', 'W1479', 'W1505', 'W1510', 'W1512', 'W1514', 'W1520']
financial distress ['W0001', 'W0506']
auditing ['W0001', 'W0163', 'W0790']
corporate governance ['W0001', 'W0003', 'W0014', 'W0485', 'W0494', 'W0500']
financial reporting ['W0001', 'W0003', 'W0790']
tax avoidance ['W0002', 'W0003']
emerging markets ['W0003', 'W0310', 'W0818']
gender equality ['W0003', 'W0999']
political connections ['W0003', 'W0502', 'W0788', 'W0802']
culture ['W0003', 'W0234', 'W0312', 'W0317', 'W0687', 'W0707', 'W0733', 'W0794', 'W0812', 'W1111', 'W1182', 'W1185', 'W1321', 'W1491', 'W1515']
mathematical programming ['W0004', 'W0155', 'W0856']
statistics ['W0004', 'W0210', 'W033

Creating a new dictionary that adds researcher names and affiliations to each of the researcherIDs. Exporting final file that lists each faculty and their departmental affiliations for each keyword.

In [12]:
#creating researcherID:researcherName,researcherAffiliation dictionary outside of loop
uniqueIDaffil_dict = dict({})

#looping through each entry within the researcherID:researcherName,researcherAffiliation dictionary
with open(researcher_affiliation, 'r', encoding="utf8") as f: #opening file
    uniqueID_name_affiliation = f.readlines() #reading in file as string
    for line in uniqueID_name_affiliation: #for each researcherID, researcherName, researchAffiliation within line
        temp_list = line.rstrip('\n').replace(';',':').split(':') #split line of string into list
        uniqueID = temp_list[0] #take the first value in list (researcherID) and put it into a temporary list
        name = temp_list[1] #take the second value in list (researcherName) and put in temporary list
        affiliation = temp_list[2] #take the third value in list (researcherAffiliation) and put in temporary list
        uniqueIDaffil_dict[uniqueID] = (name,affiliation) #create dictionary entry, setting key as researcherID and values as researcherName and researcherAffiliation

#creating keyword:researcherID,researcherName,researcherAffiliation dictionary outside of loop
keyresearaffil_dict = dict({})

#looping through each entry within the researcherID:keyword dictionary
for keyword, uniqueIDs in keyresear_dict.items():
    affiliations = [] #affiliations placeholder list
    userIDs = [] #researcherIDs placeholder list
    userNames = [] #researcherNames placeholder list
    
    #looping through each researcherID value within researcherID:keyword dictionary
    for uniqueID in uniqueIDs:
        affiliations.append(uniqueIDaffil_dict[uniqueID][1]) #add associated affiliation from within the researcherID:researcherName,researcherAffiliation dictionary to affiliations placeholder list
        userIDs.append(uniqueID) #add associated researcherID to researcherIDs placeholder list
        userNames.append(uniqueIDaffil_dict[uniqueID][0]) #add associated researcherNames from within the researcherID:researcherName,researcherAffiliation dictionary to researcherNames placeholder list
    keyresearaffil_dict[keyword] = [userIDs,userNames,list(set(affiliations))] #setting dictionary key:values, deduplicating affiliations

#creating dataframe from dictionary, where the keyword is one column, researcherIDs are second column, researcherNames are third column, and researcherAffiliations are fourth column  
keyresearaffil_dataframe = pd.DataFrame.from_dict(keyresearaffil_dict, orient='index')

#looking up today's date and exporting dataframe to comma separated value file with utf-8 encoding  
todays_date = date.today()
keyresearaffil_dataframe.to_csv(f"{todays_date.isoformat()}_WMUresearcher_keywords_affiliation.csv", index=True, encoding='utf-8-sig')